#Table of Contents
* [1. Web Scraping](#1.-Web-Scraping)
* [2. Filbert checking data](#2.-Filbert-checking-data)

In [1]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import json
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests

# checking data

In [3]:
df = pd.read_json("CS109finalproj/tempdata/movieinfo_2014_full.json")
df = df.append(pd.read_json("CS109finalproj/tempdata/movieinfo_2013.json"))
df = df.append(pd.read_json("CS109finalproj/tempdata/movieinfo_2012.json"))
df = df.append(pd.read_json("CS109finalproj/tempdata/movieinfo_2011.json"))
df = df.append(pd.read_json("CS109finalproj/tempdata/movieinfo_2010.json"))
df = df.append(pd.read_json("CS109finalproj/tempdata/movieinfo_2009.json"))

In [4]:
r_df = df[~df["user_ratings"].isnull()].reset_index()

In [5]:
len(r_df)

32788

In [6]:
new_df = r_df[r_df["location_page"]!="None"].reset_index()
new_df

,level_0,index,budget,contentRating,country,critic_ratings,duration,genre,gross,language,location_page,name,opening_weekend,release_dates,url,user_ratings,user_ratings_count,year
0,0,0,"€1,500,000",NaN,Kazakhstan,NaN,110 min,[Drama],NaN,Russian,"[Minsk, Belarus , Almaty, Kazakhstan , St. Pet...",Ya ne vernus,NaN,[ 1 March 2014 (Russia) ],/title/tt2637844/,6.9,320,2014
1,2,3,NaN,NaN,USA,NaN,85 min,"[Horror, Mystery, Thriller]",NaN,English,"[Silt, Colorado, USA ]",Find Me,NaN,[ 1 September 2014 (USA) ],/title/tt3027188/,4.5,649,2014
2,3,4,NaN,NaN,Ireland,NaN,88 min,"[Comedy, Drama, Family]",NaN,English,"[Dublin, County Dublin, Ireland , County Wickl...",Gold,NaN,[ 10 October 2014 (Ireland) ],/title/tt3134422/,6.1,406,2014
3,7,10,NaN,Not,USA,NaN,85 min,"[Action, Sci-Fi, Thriller]",NaN,English,"[Los Angeles, California, USA , Long Beach, Ca...",Mega Shark vs. Mecha Shark,NaN,[ 28 January 2014 (USA) ],/title/tt3152098/,2.6,"1,988",2014
4,8,11,NaN,NaN,Poland,NaN,117 min,"[Drama, Romance]",NaN,Polish,"[Ancona, Marche, Italy , Warsaw, Mazowieckie, ...",Obce cialo,NaN,[ 5 December 2014 (Poland) ],/title/tt3997248/,4.6,156,2014
5,12,19,"$50,000",NaN,USA,NaN,NaN,[Comedy],NaN,English,"[Angelus Oaks, California, USA , California, U...",Camp-Off,NaN,[ 4 April 2014 (USA) ],/title/tt3482042/,7.3,7,2014
6,14,21,NaN,NaN,Germany,NaN,84 min,"[Comedy, Drama]",NaN,German,"[Akademie der Künste, Berlin, Germany , Haus d...",Ich will mich nicht künstlich aufregen,NaN,[ February 2014 (Germany) ],/title/tt3471498/,6.2,28,2014
7,15,23,NaN,NaN,Sweden,NaN,99 min,"[Action, Crime, Drama]",NaN,Swedish,"[Falkenberg, Hallands län, Sweden , Uddevalla,...",Svart kung,NaN,[ 15 March 2014 (Sweden) ],/title/tt2935416/,6.2,42,2014
8,16,24,"$7,100",NaN,USA,NaN,92 min,[Action],NaN,English,"[New York, USA , Paulsboro, New Jersey, USA , ...",Battle,NaN,[ 25 December 2014 (USA) ],/title/tt2094769/,6.6,8,2014
9,20,29,NaN,NaN,UK,NaN,90 min,[Horror],NaN,English,"[Wales, UK ]",Valley of the Witch,NaN,[ 13 January 2015 (USA) ],/title/tt2908340/,5.0,67,2014


In [7]:
len(new_df)

18031

In [8]:
places = set()
places_freq = {}
separated_places = []
for row in new_df["location_page"]:
    temp = []
    for loc in row:
        split = loc.split(",")
        for item in split:
            stripped = item.encode('utf8').strip().lower()
            if stripped!="":
                places.add(stripped)
                temp.append(stripped)
                if stripped not in places_freq.keys():
                    places_freq[stripped] = 1
                places_freq[stripped] = places_freq[stripped] + 1
    separated_places.append(temp)
df_boolean = new_df.copy()
df_boolean['separated_places'] = separated_places

In [9]:
places

{'esher',
 "hell's kitchen",
 'rhineland-palatinate',
 'glamis',
 'value drugs',
 'parc de sceaux',
 'mgm grand hotel - 3799 las vegas boulevard south',
 'tilton',
 'saylorsburg',
 'stonehurst',
 'uithoorn',
 'sant just desvern',
 'mazari sharif',
 'roslyn',
 'khanua',
 'cyprus',
 'la garriga',
 'zittau',
 'laiya-aplaya',
 'luj\xc3\xa1n',
 'lillestr\xc3\xb8m',
 'brownstone street',
 'chishui',
 'kbs suwon studio',
 'thines',
 'isleton',
 'cropvale farm',
 'str\xc3\xb6mstad',
 'colorno',
 '1721 west olive ave',
 "station centrale d'autobus",
 'conejo valley',
 'keighley railway station',
 'hermann',
 'm\xc3\xa9rignac',
 'wayland',
 'saipan',
 'belvoir',
 'kannapolis',
 'puente de ixtla',
 'utrera',
 'saskatchewan',
 'manhatan',
 'grand hotel excelsior',
 'bratislava',
 'cafe formaggio',
 'the concert hall',
 'gulmarg',
 '625 stanwix tower apartments',
 'valley forge national historical park - n. gulph road',
 'roissy-en-france',
 'avondale',
 'kottayam',
 'normansfield hospital theatre'

In [10]:
len(places)

18983

In [11]:
import csv
country_list = set()
with open('CS109finalproj/tempdata/countries.csv', 'rb') as csvfile:
    countries = csv.reader(csvfile, delimiter=',')
    for country in countries:
        country_list.add(country[1].strip().lower())

#find the places that are not countries
places_no_country = places - country_list
#len(places - country_list)
#len(country_list & places) #number of countries = 176 present in both sets
#print places_no_country

In [12]:
#count  = 0

testdf = pd.DataFrame()
for place in places_no_country:
        testdf[place] = [place in location for location in df_boolean.separated_places]
        #count = count + 1
        #if count > 20:
        #    break

In [13]:
testdf.head(10)

,square des francine,rhineland-palatinate,glamis,value drugs,parc de sceaux,mgm grand hotel - 3799 las vegas boulevard south,tilton,saylorsburg,stonehurst,uithoorn,great otway national park,mazari sharif,roslyn,khanua,new beverly cinema,jim henson company studios - 1416 n. la brea avenue,revere,zittau,laiya-aplaya,luján,lillestrøm,brownstone street,chishui,47 rue de la goutte d'or,cropvale farm,strömstad,colorno,1721 west olive ave,301 loma colorado dr ne,keighley railway station,hermann,olympic car wash - 3554 west olympic boulevard,viale savolotti felice,ladera ranch,saipan,delhi,esher,puente de ixtla,saskatchewan,turkish republic of north cyprus,wilton house,bratislava,fargo,2 rue des deux gares,625 stanwix tower apartments,valley forge national historical park - n. gulph road,roissy-en-france,avondale,Årdal,fundo san miguel de colina,...,pentrebach,antonito,american falls,tama river,dilijan,sopelana,moab,callanwolde fine arts center - 980 briarcliff road ne,tredegar,trona pinnacles - 300 s. richmond road,nochixtlán,figueras,molskroen,clapham junction,chioggia,newport beach,saschiz,lleida,wellness center,westin bonaventure hotel & suites - 404 s. figueroa street,chaves,london palladium,museu romànic can papiol,wieliszew,howard st & s 11th st,shere,kinde,harefield,place du colonel fabien,vence,whitby,keswick,fort bonifacio,tai-tung,rondo dmowskiego,enontekiö,tarrytown,bortanic gardens hakgala,brechin,2605 west 5th avenue,totes gebirge,calafate,makati city,west sumatra,tremonton,gorham,heathcote,richardson,heide,creuzburg
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,F

In [14]:
len(testdf)

18031

In [15]:
testdf.astype(int)

,square des francine,rhineland-palatinate,glamis,value drugs,parc de sceaux,mgm grand hotel - 3799 las vegas boulevard south,tilton,saylorsburg,stonehurst,uithoorn,great otway national park,mazari sharif,roslyn,khanua,new beverly cinema,jim henson company studios - 1416 n. la brea avenue,revere,zittau,laiya-aplaya,luján,lillestrøm,brownstone street,chishui,47 rue de la goutte d'or,cropvale farm,strömstad,colorno,1721 west olive ave,301 loma colorado dr ne,keighley railway station,hermann,olympic car wash - 3554 west olympic boulevard,viale savolotti felice,ladera ranch,saipan,delhi,esher,puente de ixtla,saskatchewan,turkish republic of north cyprus,wilton house,bratislava,fargo,2 rue des deux gares,625 stanwix tower apartments,valley forge national historical park - n. gulph road,roissy-en-france,avondale,Årdal,fundo san miguel de colina,...,pentrebach,antonito,american falls,tama river,dilijan,sopelana,moab,callanwolde fine arts center - 980 briarcliff road ne,tredegar,trona pinnacles - 300 s. richmond road,nochixtlán,figueras,molskroen,clapham junction,chioggia,newport beach,saschiz,lleida,wellness center,westin bonaventure hotel & suites - 404 s. figueroa street,chaves,london palladium,museu romànic can papiol,wieliszew,howard st & s 11th st,shere,kinde,harefield,place du colonel fabien,vence,whitby,keswick,fort bonifacio,tai-tung,rondo dmowskiego,enontekiö,tarrytown,bortanic gardens hakgala,brechin,2605 west 5th avenue,totes gebirge,calafate,makati city,west sumatra,tremonton,gorham,heathcote,richardson,heide,creuzburg
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
testdf[testdf["glamis"]==1]

,square des francine,rhineland-palatinate,glamis,value drugs,parc de sceaux,mgm grand hotel - 3799 las vegas boulevard south,tilton,saylorsburg,stonehurst,uithoorn,great otway national park,mazari sharif,roslyn,khanua,new beverly cinema,jim henson company studios - 1416 n. la brea avenue,revere,zittau,laiya-aplaya,luján,lillestrøm,brownstone street,chishui,47 rue de la goutte d'or,cropvale farm,strömstad,colorno,1721 west olive ave,301 loma colorado dr ne,keighley railway station,hermann,olympic car wash - 3554 west olympic boulevard,viale savolotti felice,ladera ranch,saipan,delhi,esher,puente de ixtla,saskatchewan,turkish republic of north cyprus,wilton house,bratislava,fargo,2 rue des deux gares,625 stanwix tower apartments,valley forge national historical park - n. gulph road,roissy-en-france,avondale,Årdal,fundo san miguel de colina,...,pentrebach,antonito,american falls,tama river,dilijan,sopelana,moab,callanwolde fine arts center - 980 briarcliff road ne,tredegar,trona pinnacles - 300 s. richmond road,nochixtlán,figueras,molskroen,clapham junction,chioggia,newport beach,saschiz,lleida,wellness center,westin bonaventure hotel & suites - 404 s. figueroa street,chaves,london palladium,museu romànic can papiol,wieliszew,howard st & s 11th st,shere,kinde,harefield,place du colonel fabien,vence,whitby,keswick,fort bonifacio,tai-tung,rondo dmowskiego,enontekiö,tarrytown,bortanic gardens hakgala,brechin,2605 west 5th avenue,totes gebirge,calafate,makati city,west sumatra,tremonton,gorham,heathcote,richardson,heide,creuzburg
2540,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


#PCA

In [17]:
from sklearn.decomposition import PCA

pca = PCA(n_components=60)

X = pca.fit_transform(testdf)

In [19]:
pca.explained_variance_ratio_


array([ 0.08966082,  0.04085628,  0.03511292,  0.02106409,  0.00813291,
        0.00738843,  0.00702697,  0.00689784,  0.00656593,  0.00609479,
        0.00581895,  0.00570325,  0.00563239,  0.00535749,  0.00523605,
        0.00498002,  0.00490363,  0.00471621,  0.00448533,  0.0042609 ,
        0.00415812,  0.00414743,  0.0040415 ,  0.00399923,  0.00394946,
        0.00359582,  0.00347854,  0.00334254,  0.00332164,  0.00324718,
        0.00319029,  0.00314148,  0.00308985,  0.00291546,  0.00286129,
        0.00281125,  0.00278004,  0.00266625,  0.00261327,  0.00253808,
        0.00251579,  0.00249877,  0.00242453,  0.00239133,  0.0023684 ,
        0.00234974,  0.00234393,  0.0023217 ,  0.00229824,  0.00223562,
        0.00218376,  0.00196221,  0.00194116,  0.00191752,  0.00187177,
        0.00186741,  0.00183997,  0.0018061 ,  0.00180234,  0.00180082])

In [20]:
print pca.explained_variance_ratio_.sum()

0.38852503425


In [22]:
#New DF of principal components as independent variables
Xdf = pd.DataFrame()

for i in range(pca.explained_variance_ratio_.shape[0]):
    Xdf["pc%i" % (i+1)] = X[:,i]

Xdf.head()

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,pc11,pc12,pc13,pc14,pc15,pc16,pc17,pc18,pc19,pc20,pc21,pc22,pc23,pc24,pc25,pc26,pc27,pc28,pc29,pc30,pc31,pc32,pc33,pc34,pc35,pc36,pc37,pc38,pc39,pc40,pc41,pc42,pc43,pc44,pc45,pc46,pc47,pc48,pc49,pc50,pc51,pc52,pc53,pc54,pc55,pc56,pc57,pc58,pc59,pc60
0,0.458387,0.019477,0.210905,0.097086,-0.058284,0.003107,-0.020195,0.045751,-0.010622,0.050096,-0.003129,0.013653,-0.002408,0.052699,-0.040004,0.067982,-0.063430,0.027079,-0.058716,-0.019659,0.032495,-0.024102,0.259602,-0.067166,0.218821,-0.838273,0.197999,-0.085401,0.030917,-0.077174,0.118222,-0.026416,0.076915,-0.045134,0.061982,-0.003009,0.001391,-0.008625,0.003821,-0.014213,0.004527,0.005973,-0.019683,-0.007764,0.012386,0.004317,0.001377,-0.001175,-0.004597,0.010863,-0.003293,-0.001583,-0.003400,0.007400,-0.003271,-0.010135,-0.000102,0.001775,0.008908,0.005394
1,-0.314210,-0.282463,-0.000817,-0.405089,-0.012024,-0.071096,-0.015946,0.007528,-0.060495,-0.000283,-0.002982,0.008194,0.042948,-0.005579,-0.041653,0.027360,-0.011423,-0.091576,0.003548,-0.017263,0.006508,-0.020082,-0.014430,0.015567,-0.103017,-0.002574,0.110315,-0.003647,-0.010587,0.008613,0.015850,-0.044517,0.031789,-0.004527,-0.012571,-0.011857,0.002252,0.073870,0.003901,0.000266,0.036639,-0.014370,0.085645,0.014527,0.034249,0.047540,-0.010860,0.102027,0.029952,0.001488,-0.128211,0.000693,0.232122,-0.028561,0.169768,0.064934,0.052597,0.024643,-0.087129,-0.018300
2,0.455325,0.022879,0.201829,0.089696,-0.033418,0.011630,-0.020028,0.026678,-0.006662,0.045546,-0.001263,-0.001357,-0.012796,0.029302,-0.005449,0.029163,-0.041245,0.008682,-0.030933,-0.029911,-0.021623,0.006809,0.027109,0.025917,0.024156,0.032679,-0.010338,0.027823,-0.018199,0.032426,-0.033511,-0.009292,-0.050137,-0.042778,-0.021092,-0.008938,0.098811,-0.057830,-0.019072,0.047531,-0.031723,-0.015747,-0.074606,0.017054,-0.056045,0.015022,0.022238,-0.013685,0.014377,-0.117791,0.073363,0.073127,-0.063669,-0.009926,-0.066935,-0.005653,-0.000174,0.058512,-0.178121,0.353612
3,-1.145919,0.664413,0.129150,0.204244,-0.015634,0.022721,-0.020827,0.008190,0.032613,0.029775,0.065129,0.036843,0.075879,0.005341,-0.001269,0.019885,-0.012285,-0.032596,-0.008208,0.037625,-0.011452,-0.003222,-0.010853,0.016491,-0.025258,-0.012021,0.006776,0.021851,-0.005387,0.088097,0.060520,0.036112,0.006220,0.014049,0.003554,-0.005269,-0.002218,0.014636,-0.000013,0.013808,0.006614,0.018125,-0.019385,-0.007208,0.001865,0.003041,-0.008462,0.024346,0.017944,0.002223,0.001253,-0.003727,0.013602,0.012470,-0.061235,-0.022366,0.043955,-0.002163,-0.004657,-0.000034
4,0.475495,0.021888,0.225534,0.118178,-0.085471,0.007807,-0.029614,0.073982,-0.009550,0.069950,-0.009941,0.010819,-0.021749,0.099468,-0.020508,0.080441,-0.095385,-0.013252,-0.098558,-0.032792,0.050805,-0.041572,0.435017,-0.070288,0.334597,-1.222609,0.279595,-0.128890,0.087372,-0.050529,0.083701,-0.059133,-0.024942,0.306390,-0.796952,0.804231,-0.388453,0.049443,0.067218,0.031527,0.013724,0.001651,-0.006382,-0.040596,0.064891,-0.006540,-0.061771,-0.064775,0.004317,0.137427,-0.039719,-0.012241,-0.004438,0.064201,0.019075,-0.040061,0.005386,0.019828,-0.038789,-0.076414
